Imports

In [1]:
#------------------ General Purpose Imports ---------------------#
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from timeit import default_timer as timer
import pickle

#------------------ Machine Learning Imports ---------------------#
import torch
import scipy
import torch.utils.data
from sklearn.model_selection import train_test_split

#------------------ Dataset imports ---------------------#
import dysts
from dysts import flows


In [2]:
#------------------ system imports ---------------------#
import os
import sys
sys.path.append(os.path.abspath(os.path.join('../..')))

#------------------ reservoirgrid imports ---------------------#
from reservoirgrid.models import Reservoir
from reservoirgrid.helpers import utils
from reservoirgrid.helpers import viz
from reservoirgrid.helpers import chaos_utils
from reservoirgrid.helpers import reservoir_tests
#--------------------------------------------------------------#

device setting

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

## Loading Datasets

In [4]:
system_list = ["SprottA","RabinovichFabrikant", "Halvorsen", 
               "GuckenheimerHolmes", "Chua", "Chen", "MultiChen", "Rossler"]

system_name = "Chaotic/" + system_list[-1]
path = "../../reservoirgrid/datasets/" + system_name + ".npy"

# Existance check
if not os.path.exists(path):
    print(f"{system_name} does not exist, Generate First")
else:
    print(f"{system_name} exist, loading from datasets")
    system = np.load(path, allow_pickle=True)
    print("System loaded")

Chaotic/Rossler exist, loading from datasets
System loaded


## Data processing

In [5]:
T_system = utils.truncate(system) #truncated system to have same periods
pp_select = 10

input = T_system[pp_select][1] # selecting the sample 
input = utils.normalize_data(input)

Vizulizing Datasets

In [6]:
viz.plot_components(input)

In [7]:
viz.compare_plot(datasets=[input[:,:3]], figsize=(600, 500))

## Trainig and predictions

In [8]:
parameter_dict_single = {
    "SpectralRadius": [0.5],
    "LeakyRate": [0.1],
    "InputScaling": [1]
}
result_folder = "results/test/" + system_name + "/" + pp_num + ".pkl"

if not os.path.exists(result_folder):
    os.makedirs(result_folder)
    results = utils.parameter_sweep(inputs=input, parameter_dict=parameter_dict_single, 
                        reservoir_dim=1300, input_dim= 3, 
                        output_dim=3, sparsity=0.9, return_targets=True)
    pp_num = str(T_system[pp_select][0])
    result_path = result_folder
    # Saving the result
    with open(result_path , 'wb') as f:
        pickle.dump(results, f)
else:
    print("Results already exists, Loading from saved")
    result_path = result_folder  + pp_num + ".pkl"
    with open(result_path, 'rb') as f:
        results = pickle.load(f)

NameError: name 'pp_num' is not defined

In [ ]:
results = utils.parameter_sweep(inputs=input, parameter_dict=parameter_dict_single, 
                        reservoir_dim=1300, input_dim= 3, 
                        output_dim=3, sparsity=0.9, return_targets=True)

[Data preparation] elapsed: 0.01s

Combination 1/1 - SR: 0.5, LR: 0.1, IS: 1
[Model init] elapsed: 0.77s
Readout training complete.
[Training] elapsed: 0.72s
[Prediction] elapsed: 0.73s
[State extraction] elapsed: 0.01s
RMSE: 0.7427 | Iter time: 2.22s


saving the results

In [ ]:
param_num = 0
print(results[param_num]['parameters'], results[param_num]['metrics'])
predictions = results[param_num]['predictions']
test_targets = results[param_num]['true_value']
reservoir_states_downed = results[param_num]['reservoir_states']

{'SpectralRadius': 0.5, 'LeakyRate': 0.1, 'InputScaling': 1} {'RMSE': 0.28114450985272965}


In [ ]:
for_plot = test_targets[:,:3], predictions[:,:3]
viz.compare_plot(for_plot, titles=["y", "ypred"], figsize=(1080,600))
viz.plot_components(test_targets[:], line_width=1,figsize=(1080,600))
viz.plot_components(predictions[:], line_width=1,figsize=(1080,600))

In [ ]:
result_path = "results/Quasiperiodic/Torus/100.0.pkl"
with open(result_path , 'rb') as f:
    data = pickle.load(f)

In [ ]:
T_system['pp']

array([  5.,  10.,  15.,  20.,  25.,  30.,  35.,  40.,  45.,  50.,  55.,
        60.,  65.,  70.,  75.,  80.,  85.,  90.,  95., 100.])

In [ ]:
min([data[i]['metrics'] ['RMSE'] for i in range(len(data))]), np.argmin([data[i]['metrics'] ['RMSE'] for i in range(len(data))])

(0.00762863920738742, np.int64(188))

In [ ]:
select = 104
print(data[select]['parameters'])
viz.compare_plot(datasets=[data[select]['true_value'], data[select]['predictions']], titles=["true","preds"])

{'SpectralRadius': 0.9, 'LeakyRate': 0.5, 'InputScaling': 1.0}


In [ ]:
viz.plot_components(data[select]['predictions'][:500]), viz.plot_components(data[select]['true_value'][:500])

(None, None)